# Мэтчинг товаров маркетплейса

Сопоставление или “мэтчинг” (англ. matching - соответствия) - одна из базовых задач машинного обучения, которая встречается в информационном поиске, компьютерном зрении, рекомендательных системах и др.

Имеются реальные сырые данные от одного из крупнейших маркетплейсов страны. Задача - сопоставить и найти наиболее похожие товары. Необходимо:

🔸 разработать алгоритм, который для всех товаров из `validation.csv` предложит несколько вариантов наиболее похожих товаров из `base.csv`  
🔸 оценить качество алгоритма по метрике `accuracy@5`

Импортируем полезные библиотеки, которые пригодятся нам в дальнейшем

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
from sklearn.base import BaseEstimator
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
from lightgbm import LGBMRegressor

Зададим параметры по умолчанию и объявим константы, которые понадобятся нам в дальнейшем

In [ ]:
RANDOM_STATE = 751286
SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

pd.set_option('display.precision', 4)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.rc('font', size=SMALL_SIZE)                                    # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)                              # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)                              # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)                              # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)                            # fontsize of the figure title
plt.rc('figure', figsize=(24, 18))                                 # size of the figure

## Подготовка данных


### Загрузка и разведочный анализ данных

#### Получим данные с информацией о товарах

In [ ]:
# Для экономии памяти будем использовать np.float32 вместо np.float64
dtypes = {str(i): np.float32 for i in range(72)}
dtypes.update({'Id': 'object', 'Target': 'object'})

base = pd.read_csv('./base.csv', dtype=dtypes)
train = pd.read_csv('./train.csv', dtype=dtypes)
validation = pd.read_csv('./validation.csv', dtype=dtypes)

#### Ознакомимся с набором данных. Выведем несколько строк из датафреймов

In [ ]:
display(base.sample(5, random_state=RANDOM_STATE))
display(train.sample(5, random_state=RANDOM_STATE))
display(validation.sample(5, random_state=RANDOM_STATE))

#### Отобразим информацию для краткого обзора данных:

In [ ]:
base.info()
train.info()
validation.info()

#### Посчитаем количество пропущенных значений:

In [ ]:
print(f'Количество пропусков в датафрейме base: {base.isna().sum().sum()}')
print(f'Количество пропусков в датафрейме train: {train.isna().sum().sum()}')
print(f'Количество пропусков в датафрейме validation: {validation.isna().sum().sum()}')

#### Отобразим таблицу с описательной статистикой признаков:

In [ ]:
display(base.describe())
display(train.describe())
display(validation.describe())

#### Отобразим таблицу с попарными корреляциями признаков:

In [ ]:
# Воспользуемся сэмплирование для того, чтобы уменьшить потребление ресурсов
base_corr = base.sample(frac=0.5).corr()
train_corr = train.corr()
validation_corr = validation.corr()

display(base_corr.style.background_gradient(axis=None, cmap='Blues'))
display(train_corr.style.background_gradient(axis=None, cmap='Blues'))
display(validation_corr.style.background_gradient(axis=None, cmap='Blues'))

In [ ]:
tol = 0.18
size = 72
base_high_corr = (base_corr > tol).sum().sum() - size
train_high_corr = (train_corr > tol).sum().sum() - size
validation_high_corr = (validation_corr > tol).sum().sum() - size

print(f'Количество недиагональных попарных корреляций, значения которых больше чем {tol} в base: {base_high_corr}')
print(f'Количество недиагональных попарных корреляций, значения которых больше чем {tol} в train: {train_high_corr}')
print(f'Количество недиагональных попарных корреляций, значения которых больше чем {tol} в validation: {validation_high_corr}')

#### Отобразим гистограммы распределений значений признаков:

In [ ]:
print('гистограммы для датафрейма base:')
base.hist()
plt.tight_layout()
plt.show()

print('гистограммы для датафрейма train:')
train.hist()
plt.tight_layout()
plt.show()

print('гистограммы для датафрейма validation:')
validation.hist()
plt.tight_layout()
plt.show()

#### Краткий вывод:
- в данных присутствуют нарушения правила хорошего стиля в названиях признаков
- в данных отсутствуют несоответствия типов в признаках 
- в данных отсутствуют пропуски в признаках
- в данных присутствуют аномальные значения
- в данных отсутствуют намёки на мультиколлинеарность
- практически все признаки имеют распределение близкое к нормальному распределению

### Предобработка данных

#### Исправим нарушения правил хорошего стиля в названиях столбцов:

In [ ]:
base.rename(inplace=True, columns={str(i): 'feature_'+str(i) for i in range(72)} | {'Id': 'id'})
train.rename(inplace=True, columns={str(i): 'feature_'+str(i) for i in range(72)} | {'Id': 'id', 'Target': 'target'})
validation.rename(inplace=True, columns={str(i): 'feature_'+str(i) for i in range(72)} | {'Id': 'id'})

#### Удалим неинформативные признаки, которые не несут ценности для прогноза:

In [ ]:
base.drop(['id'], axis=1, inplace=True)
train.drop(['id'], axis=1, inplace=True)
validation.drop(['id'], axis=1, inplace=True)

#### Очистим данные от дубликатов

In [ ]:
# check
print(base.duplicated().sum())
print(train.duplicated().sum())
print(validation.duplicated().sum())

#### Очистим данные от аномальных значений

Todo

#### Создадим базовый пайплайн с предобработкой данных

Todo

#### Краткий вывод:
Todo

## Обучение моделей

In [14]:
# Модель, которая разделяет данные на кластеры и для каждого кластера обучает свою модель KNN
class NearestNeighborsClusterMix(BaseEstimator):
    def __init__(self, batch_size=1024, **kwargs):
        self.batch_size = batch_size
        self.kmeans = MiniBatchKMeans(**kwargs)
        self.models = []
        # Таблица с информацией о номере кластера, в котором находится объект, и его индекс внутри этого кластера
        self.table = pd.DataFrame([], columns=['cluster', 'cluster_index'])
        
    def get_index(self, cluster, cluster_index):
        return self.table[(self.table.cluster==cluster) & (self.table.cluster_index==cluster_index)].index[0]
    
    def fit(self, X):
        for chunk in np.array_split(X, self.batch_size):
            self.kmeans.partial_fit(chunk)
        
        cluster_nums = []
        for chunk in np.array_split(X, self.batch_size):
            cluster_nums.extend(self.kmeans.predict(chunk))
        self.table.cluster = cluster_nums
            
        for i in self.table.cluster.sort_values().unique():
            indexes = self.table.cluster[self.table.cluster==i].index.values
            cluster_chunk = X.loc[indexes]
            self.models.append(NearestNeighbors().fit(cluster_chunk))
            #self.table.loc[indexes, 'cluster_index'] = list(itertools.chain(*self.models[i].kneighbors(cluster_chunk, n_neighbors=1)[1]))
            
    def predict_cluster_(self, X):
        return self.kmeans.predict(X)

    def predict(self, X):
        predictions = []
        cluster_nums = self.predict_cluster_(X)
        for i, row in X.iterrows():
            indexes = self.models[cluster_nums[i]].kneighbors(X.iloc[i:i+1], n_neighbors=5)[1][0]
            #predictions.append([self.get_index(cluster_nums[i], index) for index in indexes])
        return predictions

In [15]:
%%time
nncm = NearestNeighborsClusterMix(batch_size=1024, n_clusters=10)
nncm.fit(base)

CPU times: user 14.9 s, sys: 1.96 s, total: 16.8 s
Wall time: 16.8 s


In [16]:
nncm.predict(validation)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# TODO
# Общий вывод
# Оптимизировать класс NearestNeighborsClusterMix
# Сделать пайплайн
# Оптимизировать гиперпараметры при помощи кросс-валидации
# Анализ результатов



# Это фиаско